# Milestone 2
Frederik Kallestrup Mastratis (qln174)  
Dongyu Liu (dlf327)  
Shamim Tariq Akram (zmx145)  
Celina Aurora Nguyen (szf345)  

Group 21

## Task 1



A(_id_, domain, type, url, content, timestamps, title, summary)

T(_id_:int, _tagid_:string)

AU(_id_:int, _name_:string)

K(_id_, _keyword_:string)

MK(_id_, _keyword_:string)






In [ ]:
import pandas as pd
import numpy as np
import psycopg2

df_load = pd.read_csv("1mio-raw.csv", delimiter = ",", nrows = 10000)

In [ ]:
def applyalong(df, column, f):
    df[column] = df[column].apply(f)

# Splits a string into list
def string_splitter(string):
    lst = str(string).split(", ")
    filter_obj = filter(lambda x: x != "", lst)
    return list(filter_obj)

# Strip a string representation of list of strings
def string_stripper(string):
    lst = [i.strip() for i in string[1:-1].replace('\'',"").split(',')]
    filter_obj = filter(lambda x: x != "", lst)
    return list(filter_obj)

def string_filter(lst):
    filters = [lambda x: not x.isdigit(), lambda x: x != ""]
    filter_obj = filter(lambda x: all([f(x) for f in filters]), lst)
    return list(filter_obj)

df = df_load.copy()

# Dropping columns (setting new ID column later)
df = df.drop(columns = ['Unnamed: 0', 'id', 'source'])

# Set new ID column
df = df.rename_axis('id').reset_index()
df.set_index('id')

df = df.astype({'domain':str, 'type':str, 'url':str, 'content':str, 'scraped_at':str, 'inserted_at':str,
        'updated_at':str, 'title':str, 'authors':str, 'keywords':str, 'meta_keywords':str,
        'meta_description':str, 'tags':str, 'summary':str}, copy = False)

# Convert blank fields into NaN
#df = df.replace(r'^\s*$', np.nan, regex=True)

# Replace 'nan' strings with NaN
df = df.replace("nan", np.nan)

# Convert all strings into lower case:
df = df.applymap(lambda s: s.lower() if type(s) == str else s)

# Clean types
type_set = ['fake', 'satire', 'bias', 'conspiracy', 'state', 'junksci', 'hate', 'clickbait', 'unreliable', 'political', 'reliable','rumor']
df['type'] = df['type'].apply(lambda x: np.nan if x not in type_set else x)



# Clean timestamps
#map(lambda t: applyalong(df, t, lambda x: pd.to_datetime(x, errors='coerce')), ['scraped_at','inserted_at','updated_at'])
for column in ['scraped_at','inserted_at','updated_at']:
    df[column] = df[column].apply(lambda x: pd.to_datetime(x, errors='coerce'))

# Clean auhtors - separate into list of strings
df['authors'] = df['authors'].apply(lambda x: string_splitter(x) if pd.notnull(x) else x)

# Clean metakeywords - strip a string representation of list of strings
df['meta_keywords'] = df['meta_keywords'].apply(string_stripper)
#df['meta_keywords'] = df['meta_keywords'].apply(lambda x: np.nan if (isinstance(x, list) and len(x)==0) else x)

# Clean tags
df['tags'] = df['tags'].apply(lambda x: string_splitter(x) if pd.notnull(x) else x)
df['tags'] = df['tags'].apply(lambda x: string_filter(x) if isinstance(x, list) else x)
#df['tags'] = df['tags'].apply(lambda x: np.nan if (isinstance(x, list) and len(x)==0) else x)

# Replace NaN into empty lists
for column in ['authors', 'keywords', 'meta_keywords', 'tags']:
    df[column] = df[column].fillna("").apply(list)


In [ ]:
print(type(df["authors"]))
df['scraped_at'].to_csv('test.csv', index=False)
display(df)


In [ ]:
tables = ["artikel", "author", "tags", "keywords", "metakeywords"]

create_tables_all = [
    """ 
    CREATE TABLE artikel (
        id INT, 
        domain VARCHAR, 
        type VARCHAR, 
        url VARCHAR, 
        content VARCHAR, 
        scraped_at TIMESTAMP, 
        inserted_at TIMESTAMP,
        updated_at TIMESTAMP, 
        title VARCHAR (256), 
        meta_description VARCHAR, 
        summary VARCHAR,

        PRIMARY KEY (id) 
    );
    """
    ,
    """
    CREATE TABLE author (
        a_id INT,
        author VARCHAR,
        PRIMARY KEY (a_id, author),
        FOREIGN KEY (a_id)
            REFERENCES artikel (id)
            ON UPDATE CASCADE ON DELETE CASCADE
    );
    """
    ,
    """ 
    CREATE TABLE tags (
        a_id INT, 
        tag VARCHAR, 
        PRIMARY KEY (a_id, tag),
        FOREIGN KEY (a_id)
            REFERENCES artikel (id)
            ON UPDATE CASCADE ON DELETE CASCADE
    );
    """
    ,
    """
    CREATE TABLE keywords (
        a_id INT, 
        keyword VARCHAR,
        PRIMARY KEY (a_id, keyword),
        FOREIGN KEY (a_id)
            REFERENCES artikel (id)
            ON UPDATE CASCADE ON DELETE CASCADE
    );
    """
    ,
    """
    CREATE TABLE metakeywords (
        a_id INT, 
        mkeyword VARCHAR,
        PRIMARY KEY (a_id, mkeyword),
        FOREIGN KEY (a_id)
            REFERENCES artikel (id)
            ON UPDATE CASCADE ON DELETE CASCADE
    );
    """
]

conn = psycopg2.connect(dbname="fakenewsdb", user="postgres", password="1234")

cursor = conn.cursor()

for table in tables:
    cursor.execute("DROP TABLE IF EXISTS " + table + " CASCADE;")

for sql in create_tables_all:
    cursor.execute(sql)

conn.commit

In [ ]:
def projectrow2tuple(fields, row):
    return tuple(map(lambda f: row[f], fields))

def insertstring(table, n):
    return "INSERT INTO {} VALUES ({}) ON CONFLICT DO NOTHING".format(table, ", ".join(map(lambda _: '%s', range(n))))

def multi_insert(server, a_id, insert, xs):
    for x in xs:
        server.execute(insert, (a_id, x))
    

def insert_csv2rows(server, csv_row):
    A_domain = ['id', 'domain', 'type', 'url', 'content', 'scraped_at',
       'inserted_at', 'updated_at', 'title', 'meta_description', 'summary']
    AU_domain = ['id', 'authors']
    T_domain = ['id', 'tags']
    K_domain = ['id', 'keywords']
    MK_domain = ['id', 'meta_keywords']
    
    Atuple = projectrow2tuple(A_domain, csv_row)
    (a_id, tags) = projectrow2tuple(T_domain, csv_row)
    (_, au) = projectrow2tuple(AU_domain, csv_row)
    (_, kws) = projectrow2tuple(K_domain, csv_row)
    (_, mkws) = projectrow2tuple(MK_domain, csv_row)

    Ainsert = insertstring("artikel", len(Atuple))
    Tinsert = insertstring("tags", 2)
    AUinsert = insertstring("author", 2)
    Kinsert = insertstring("keywords", 2)
    MKinsert = insertstring("metakeywords", 2)

    server.execute(Ainsert, Atuple)
    insert = lambda ins, xs: multi_insert(server, a_id, ins, xs)
    insert(Tinsert, tags)
    insert(AUinsert, au)
    insert(Kinsert, kws)
    insert(MKinsert, mkws)


for _, row in df.iterrows():
    insert_csv2rows(cursor, row)

conn.commit

In [ ]:
cursor.execute("SELECT * FROM author;")
cursor.fetchone()

In [ ]:
# QUERY 1

query1 = """
    SELECT distinct domain, type, scraped_at 
    FROM artikel
    WHERE type = 'reliable' AND scraped_at >= '2018-01-15'
"""

cursor.execute(query1)
cursor.fetchall()

In [ ]:
# QUERY 2

query2 = """
    SELECT author
    FROM (
        SELECT COUNT(a_id) AS cnt, author 
        FROM author 
        WHERE a_id IN (
            SELECT id
            FROM artikel
            WHERE type = 'fake'
        ) 
        GROUP BY author
    ) AS x
    WHERE cnt = (SELECT MAX(cnt) FROM x)
"""

# This gives a max, but doesnt take into account if multiple authors write max number of fake articles
query2_pseudomax = """
    SELECT COUNT(a_id), author 
    FROM author 
    WHERE a_id IN (
        SELECT id
        FROM artikel
        WHERE type = 'fake'
    )
    GROUP BY author ORDER BY COUNT(a_id) DESC LIMIT 1
"""

cursor.execute('rollback')
cursor.execute(query2_pseudomax)
cursor.fetchall()